In [1]:
import os
import shutil
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import timm

import matplotlib.pyplot as plt # For data viz
import sys
from tqdm.notebook import tqdm
import joblib

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Torchvision version', torchvision.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

C:\Users\Muhammad.Taha2\.conda\envs\ibex-updated\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


System Version: 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]
PyTorch version 2.1.0
Torchvision version 0.15.2a0
Numpy version 1.25.2
Pandas version 2.0.3


In [2]:
class SimpleImageClassifer(nn.Module):
    def __init__(self, num_classes=6):
        super(SimpleImageClassifer, self).__init__()
        # Where we define all the parts of the model
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 1280
        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),  # Dropout layer added here
            nn.Linear(enet_out_size, num_classes)
        )
    
    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
class ImagesDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

In [5]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

valid_folder = './Dataset/Validate/'
val_dataset = ImagesDataset(valid_folder, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [6]:
loaded_model = joblib.load('Image_Classification_Model.pkl')


# Load and preprocess the image
def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert("RGB")
    return image, transform(image).unsqueeze(0)

# Assuming you have a DataLoader for the validation set named 'val_loader'
# and you have the ground truth labels in 'val_labels'

# Set the model to evaluation mode
loaded_model.eval()

# Lists to store predictions and true labels
all_predictions = []
all_true_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = loaded_model(inputs)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)

        # Get predicted labels
        predictions = torch.argmax(probabilities, dim=1).cpu().numpy()

        # Store predictions and true labels
        all_predictions.extend(predictions)
        all_true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_true_labels, all_predictions)
f1 = f1_score(all_true_labels, all_predictions, average='weighted')
precision = precision_score(all_true_labels, all_predictions, average='weighted')
recall = recall_score(all_true_labels, all_predictions, average='weighted')

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 0.5632
F1 Score: 0.5559
Precision: 0.6115
Recall: 0.5632
